In [83]:
import os
import numpy as np
import pandas as pd
import random as rn
import glob
import re
import sys
import bs4

#######################
#  Tile conversion    #
#######################

def tileConvert(tile):
    #
    def asmlt(tile,eStr):
        counter = 1
        if tile == 16:
            eStr = "D5" + eStr
        else:
            for i in range(0,36,4): 
                #print("eStr",eStr)
                if tile in range(i+0,i+4):
                    eStr = str(counter) + eStr
                counter += 1
        return eStr
    #
    eStr = ""
    if tile < 36:
        eStr += "m"
        eStr = asmlt(tile,eStr)
    elif tile >= 36 and tile < 72:
        eStr += "p"
        tile -= 36
        eStr = asmlt(tile,eStr)
    elif tile >= 72 and tile < 108:
        eStr += "s"
        tile -= 72
        eStr = asmlt(tile,eStr)
    elif tile in range(108,112):
        eStr = "east"
    elif tile in range(112,116):
        eStr = "south" 
    elif tile in range(116,120):
        eStr = "west" 
    elif tile in range(120,124):
        eStr = "north"
    elif tile in range(124,128):
        eStr = "haku"
    elif tile in range(128,132):
        eStr = "hatsu" 
    elif tile in range(132,136):
        eStr = "chun"
    #
    return eStr

def numbaConvert(x):
    if x in range(0,4):
        y = 1
    elif x in range(4,8):
        y = 2
    elif x in range(8,12):
        y = 3
    elif x in range(12,16):
        y = 4
    elif x in range(16,20):
        y = 5
    elif x in range(20,24):
        y = 6
    elif x in range(24,28):
        y = 7
    elif x in range(28,32):
        y = 8
    elif x in range(32,36):
        y = 9
    #
    elif x in range(36,40):
        y = 21
    elif x in range(40,44):
        y = 22
    elif x in range(44,48):
        y = 23
    elif x in range(48,52):
        y = 24
    elif x in range(52,56):
        y = 25
    elif x in range(56,60):
        y = 26
    elif x in range(60,64):
        y = 27
    elif x in range(64,68):
        y = 28
    elif x in range(68,72):
        y = 29
    #
    elif x in range(72,76):
        y = 41
    elif x in range(76,80):
        y = 42
    elif x in range(80,84):
        y = 43
    elif x in range(84,88):
        y = 44
    elif x in range(88,92):
        y = 45
    elif x in range(92,96):
        y = 46
    elif x in range(96,100):
        y = 47
    elif x in range(100,104):
        y = 48
    elif x in range(104,108):
        y = 49
    #
    elif x in range(108,112):
        y = 60
    elif x in range(112,116):
        y = 70
    elif x in range(116,120):
        y = 80
    elif x in range(120,124):
        y = 90
    elif x in range(124,128):
        y = 100
    elif x in range(128,132):
        y = 110
    elif x in range(132,136):
        y = 120
    #
    return y

def indicatorCycleConverted(doraInd):
    '''cycle the dora indicator to the dora it is meant to indicate'''
    if doraInd in range(1,9):
        dora = doraInd + 1
    elif doraInd == 9:
        dora = 1
    elif doraInd in range(21,29):
        dora = doraInd + 1
    elif doraInd == 29:
        dora = 21
    elif doraInd in range(41,49):
        dora = doraInd + 1
    elif doraInd == 49:
        dora = 41
    elif doraInd == 60:
        dora = 70
    elif doraInd == 70:
        dora = 80
    elif doraInd == 80:
        dora = 90
    elif doraInd == 90:
        dora = 60
    elif doraInd == 100:
        dora = 110
    elif doraInd == 110:
        dora = 120
    elif doraInd == 120:
        dora = 100
    return dora

####################################################################
#     below functions taken from:
# https://github.com/MahjongRepository/tenhou-python-bot
####################################################################

def debug(args):
	#print(args)
	pass

def parse_call(code):
	# python struct only supports bytes, not bits
	# Test call: 45577, pon west
	#	= type 89, tile 116 + (1,2,3) (= west - so it works...)
	# 57423 = chi, 567s (called the 7 from player to the left - player 1?)
	#	= 0xe04f
	#	= 0b1110000001001111
	# 48745 = pon haku
	# 42537 = pon east
	# 46335 = chi 234s (called 2)
	# 29935 = chi 345p (called 5p)
	debug("Dealing with call %d = %x" % (code, code))
	call = {}
	# branching based on the JS sample code
	if code & (1 << 2):
		# chi. also matches nuki, but ignoring 3ma
		call['dealer'] = code & 0x3
		base = (code >> 10) & 0x3f
		bn = (((base/3)/7)*9+((base/3)%7))*4
		tile1 = (code >> 3) & 0x3
		tile2 = ((code >> 5) & 0x3) + 4
		tile3 = ((code >> 7) & 0x3) + 8
		# actual tile that got called is base % 3
		call['called'] = base % 3
		# Order so that the called tile is at the front
		if call['called'] == 0:
			call['tiles'] = (bn + tile1, bn + tile2, bn + tile3)
		elif call['called'] == 1:
			call['tiles'] = (bn + tile2, bn + tile1, bn + tile3)
		elif call['called'] == 2:
			call['tiles'] = (bn + tile3, bn + tile1, bn + tile2)

		call['type'] = 'chi'

		debug("Chi: %s %s %s" % call['tiles'])
	elif code & (1 << 3): # 0b111 xx = 0b010 xx
		debug("Pon")
		call['dealer'] = code & 0x3 # this is _relative to the caller_
		unused = (code >> 5) & 0x3 # the tile not in the meld
		tiles = code >> 9
		call['called'] = tiles % 3
		base = (tiles / 3.) * 4

		call['tiles'] = []
		for i in range (0,4):
			if i == unused:
				continue
			call['tiles'].append(base + i)
		call['type'] = 'pon'
	elif code & (1 << 4):
		# chakan - extending a koutsu
		# will need support logic in player to handle this
		pass
	elif code & (1 << 5):
		# nuki, NFA
		return {}
	else:
		# other kan
		debug("Kan")
		# kan
		tile = code >> 8
		call['called'] = tile % 4
		tile &= ~3 # mask out the called bits
		call['tiles'] = range(tile, tile+4)
		call['dealer'] = code & 0x3
		call['type'] = 'kan'
	try:
		call['type'] += "a"
		call['type'] = call['type'][:-1]
	except:
		call['type'] = 'none'
	return call

####################################################################
        
#renpath = os.path.abspath(os.getcwd()) + '/gameData'
renpath = os.path.abspath(os.getcwd()) + '/ultraVigor'

hands = []
rows = []
#    "actor","right","across","left",
#moveType can be "draw", "call","discard","reach","ron","tsumo

#DON'T FORGET TO INCLUDE VISIBLE DORA

columnHeaders = ["gameLogID","lobby","moveType","unconvertedTile","convertedTile","discDrawn",
                 "handNumber","moveNumber","roundWind","roundNumber","bonusSticks",
                 "unconvertedDoraIndicator","convertedDoraIndicator","convertedDora",
                 #"actorHandWind","rightHandWind","acrossHandWind","leftHandWind",
                 "actorWind",#"actorID",
                 "actorRating","rightRating","acrossRating","leftRating",
                 "actorRank","rightRank","acrossRank","leftRank",
                 "actorScore","rightScore","acrossScore","leftScore",
                 "actorHandConverted","actorHandUnconverted","rightHandConverted","rightHandUnconverted",
                 "acrossHandConverted","acrossHandUnconverted","leftHandConverted","leftHandUnconverted",
                 # actor's hand
#                  "actorHand[1m]","actorHand[2m]","actorHand[3m]","actorHand[4m]","actorHand[D5m]",
#                  "actorHand[5m]","actorHand[6m]","actorHand[7m]","actorHand[8m]","actorHand[9m]",
#                  "actorHand[1p]","actorHand[2p]","actorHand[3p]","actorHand[4p]","actorHand[D5p]",
#                  "actorHand[5p]","actorHand[6p]","actorHand[7p]","actorHand[8p]","actorHand[9p]",
#                  "actorHand[1s]","actorHand[2s]","actorHand[3s]","actorHand[4s]","actorHand[D5s]",
#                  "actorHand[5s]","actorHand[6s]","actorHand[7s]","actorHand[8s]","actorHand[9s]",
#                  "actorHand[east]","actorHand[south]","actorHand[west]","actorHand[north]","actorHand[haku]",
#                  "actorHand[hatsu]","actorHand[chun]",
#                  # right's hand
#                  "rightHand[1m]","rightHand[2m]","rightHand[3m]","rightHand[4m]","rightHand[D5m]",
#                  "rightHand[5m]","rightHand[6m]","rightHand[7m]","rightHand[8m]","rightHand[9m]",
#                  "rightHand[1p]","rightHand[2p]","rightHand[3p]","rightHand[4p]","rightHand[D5p]",
#                  "rightHand[5p]","rightHand[6p]","rightHand[7p]","rightHand[8p]","rightHand[9p]",
#                  "rightHand[1s]","rightHand[2s]","rightHand[3s]","rightHand[4s]","rightHand[D5s]",
#                  "rightHand[5s]","rightHand[6s]","rightHand[7s]","rightHand[8s]","rightHand[9s]",
#                  "rightHand[east]","rightHand[south]","rightHand[west]","rightHand[north]","rightHand[haku]",
#                  "rightHand[hatsu]","rightHand[chun]",
#                  # across's hand
#                  "acrossHand[1m]","acrossHand[2m]","acrossHand[3m]","acrossHand[4m]","acrossHand[D5m]",
#                  "acrossHand[5m]","acrossHand[6m]","acrossHand[7m]","acrossHand[8m]","acrossHand[9m]",
#                  "acrossHand[1p]","acrossHand[2p]","acrossHand[3p]","acrossHand[4p]","acrossHand[D5p]",
#                  "acrossHand[5p]","acrossHand[6p]","acrossHand[7p]","acrossHand[8p]","acrossHand[9p]",
#                  "acrossHand[1s]","acrossHand[2s]","acrossHand[3s]","acrossHand[4s]","acrossHand[D5s]",
#                  "acrossHand[5s]","acrossHand[6s]","acrossHand[7s]","acrossHand[8s]","acrossHand[9s]",
#                  "acrossHand[east]","acrossHand[south]","acrossHand[west]","acrossHand[north]","acrossHand[haku]",
#                  "acrossHand[hatsu]","acrossHand[chun]",
#                  # left's hand
#                  "leftHand[1m]","leftHand[2m]","leftHand[3m]","leftHand[4m]","leftHand[D5m]",
#                  "leftHand[5m]","leftHand[6m]","leftHand[7m]","leftHand[8m]","leftHand[9m]",
#                  "leftHand[1p]","leftHand[2p]","leftHand[3p]","leftHand[4p]","leftHand[D5p]",
#                  "leftHand[5p]","leftHand[6p]","leftHand[7p]","leftHand[8p]","leftHand[9p]",
#                  "leftHand[1s]","leftHand[2s]","leftHand[3s]","leftHand[4s]","leftHand[D5s]",
#                  "leftHand[5s]","leftHand[6s]","leftHand[7s]","leftHand[8s]","leftHand[9s]",
#                  "leftHand[east]","leftHand[south]","leftHand[west]","leftHand[north]","leftHand[haku]",
#                  "leftHand[hatsu]","leftHand[chun]",
                 #
                 ##
                 #ponds n stuff
                 "actorUnconvertedPond","actorConvertedPond",
                 "rightUnconvertedPond","rightConvertedPond",
                 "acrossUnconvertedPond","acrossConvertedPond",
                 "leftUnconvertedPond","leftConvertedPond",
                 #
                 ##
                 #actor
#                  "actorPond[1m]","actorPond[2m]","actorPond[3m]","actorPond[4m]","actorPond[D5m]",
#                  "actorPond[5m]","actorPond[6m]","actorPond[7m]","actorPond[8m]","actorPond[9m]",
#                  "actorPond[1p]","actorPond[2p]","actorPond[3p]","actorPond[4p]","actorPond[D5p]",
#                  "actorPond[5p]","actorPond[6p]","actorPond[7p]","actorPond[8p]","actorPond[9p]",
#                  "actorPond[1s]","actorPond[2s]","actorPond[3s]","actorPond[4s]","actorPond[D5s]",
#                  "actorPond[5s]","actorPond[6s]","actorPond[7s]","actorPond[8s]","actorPond[9s]",
#                  "actorPond[east]","actorPond[south]","actorPond[west]","actorPond[north]",
#                  "actorPond[haku]","actorPond[hatsu]","actorPond[chun]",
#                  #right
#                  "rightPond[1m]","rightPond[2m]","rightPond[3m]","rightPond[4m]","rightPond[D5m]",
#                  "rightPond[5m]","rightPond[6m]","rightPond[7m]","rightPond[8m]","rightPond[9m]",
#                  "rightPond[1p]","rightPond[2p]","rightPond[3p]","rightPond[4p]","rightPond[D5p]",
#                  "rightPond[5p]","rightPond[6p]","rightPond[7p]","rightPond[8p]","rightPond[9p]",
#                  "rightPond[1s]","rightPond[2s]","rightPond[3s]","rightPond[4s]","rightPond[D5s]",
#                  "rightPond[5s]","rightPond[6s]","rightPond[7s]","rightPond[8s]","rightPond[9s]",
#                  "rightPond[east]","rightPond[south]","rightPond[west]","rightPond[north]",
#                  "rightPond[haku]","rightPond[hatsu]","rightPond[chun]",
#                  #across
#                  "acrossPond[1m]","acrossPond[2m]","acrossPond[3m]","acrossPond[4m]","acrossPond[D5m]",
#                  "acrossPond[5m]","acrossPond[6m]","acrossPond[7m]","acrossPond[8m]","acrossPond[9m]",
#                  "acrossPond[1p]","acrossPond[2p]","acrossPond[3p]","acrossPond[4p]","acrossPond[D5p]",
#                  "acrossPond[5p]","acrossPond[6p]","acrossPond[7p]","acrossPond[8p]","acrossPond[9p]",
#                  "acrossPond[1s]","acrossPond[2s]","acrossPond[3s]","acrossPond[4s]","acrossPond[D5s]",
#                  "acrossPond[5s]","acrossPond[6s]","acrossPond[7s]","acrossPond[8s]","acrossPond[9s]",
#                  "acrossPond[east]","acrossPond[south]","acrossPond[west]","acrossPond[north]",
#                  "acrossPond[haku]","acrossPond[hatsu]","acrossPond[chun]",
#                  #left
#                  "leftPond[1m]","leftPond[2m]","leftPond[3m]","leftPond[4m]","leftPond[D5m]",
#                  "leftPond[5m]","leftPond[6m]","leftPond[7m]","leftPond[8m]","leftPond[9m]",
#                  "leftPond[1p]","leftPond[2p]","leftPond[3p]","leftPond[4p]","leftPond[D5p]",
#                  "leftPond[5p]","leftPond[6p]","leftPond[7p]","leftPond[8p]","leftPond[9p]",
#                  "leftPond[1s]","leftPond[2s]","leftPond[3s]","leftPond[4s]","leftPond[D5s]",
#                  "leftPond[5s]","leftPond[6s]","leftPond[7s]","leftPond[8s]","leftPond[9s]",
#                  "leftPond[east]","leftPond[south]","leftPond[west]","leftPond[north]",
#                  "leftPond[haku]","leftPond[hatsu]","leftPond[chun]",
                 #
                 ##
                 ############ vv  later feature engineering  vv #############################
                 #
                 #"actorPondL3CD","rightPondL3CD","acrossPondL3CD","leftPondL3CD",
                 #"actorPondL4CD","rightPondL4CD","acrossPondL4CD","leftPondL4CD",
                 #"actorPondL5CD","rightPondL5CD","acrossPondL5CD","leftPondL5CD",
                 #
                 "actorReachStatus","rightReachStatus","acrossReachStatus","leftReachStatus",
                 #
                 ##
                 #
                 "actorIsOpen","rightIsOpen","acrossIsOpen","leftIsOpen",
                 #
                 "actorOpenTiles","rightOpenTiles","acrossOpenTiles","leftOpenTiles",
                 #FEATURE ENGINEERING
                 "actorPondCount","rightPondCount","acrossPondCount","leftPondCount","whosTurn"]
                 #"actorTenpai","rightTenpai","acrossTenpai","leftTenpai"]
                 #
                 #"actorOpenMeld1","actorOpenMeld2","actorOpenMeld3","actorOpenMeld4",
                 #"rightOpenMeld1","rightOpenMeld2","rightOpenMeld3","rightOpenMeld4",
                 #"acrossOpenMeld1","acrossOpenMeld2","acrossOpenMeld3","acrossOpenMeld4",
                 #"leftOpenMeld1","leftOpenMeld2","leftOpenMeld3","leftOpenMeld4",
                 #
                 ## FEATURE ENGINEERING MOARRRRRRRRRRRRRRRRR
                 #
#                  "actorPondManStatus","actorPondPinStatus","actorPondSouStatus","actorPondHonorStatus",
#                  "rightPondManStatus","rightPondPinStatus","rightPondSouStatus","rightPondHonorStatus",
#                  "acrossPondManStatus","acrossPondPinStatus","acrossPondSouStatus","acrossPondHonorStatus",
#                  "leftPondManStatus","leftPondPinStatus","leftPondSouStatus","leftPondHonorStatus",
#                  #
#                  "actorPondManCount","actorPondPinCount","actorPondSouCount","actorPondHonorCount",
#                  "rightPondManCount","rightPondPinCount","rightPondSouCount","rightPondHonorCount",
#                  "acrossPondManCount","acrossPondPinCount","acrossPondSouCount","acrossPondHonorCount",
#                  "leftPondManCount","leftPondPinCount","leftPondSouCount","leftPondHonorCount"

def lobbyBallparker(danRanks):
    avg = np.mean(danRanks)
    if avg < 10:
        return "main"
    if avg >= 10 and avg < 13:
        return "dan"
    if avg >= 13 and avg < 16:
        return "upper"
    else:
        return "houou"
    
    
    
    
    
    
    
    
    
    
    
#########################################################################################################
    
#import time    

q = 0
maxq = 50
#time_start = time.time()

#dfList = []

for filename in sorted(glob.glob(os.path.join(renpath, '*.maj')),reverse=True): #reverse=False for earlier games.
    if q == maxq:
        break
    if q != maxq:
        source = open(filename,'r').read()
        #os.remove(filename)
        q += 1
    if "/><BYE" in source:
        print("contained bye so dropped it")
        #pass
    else:
        print("filename",filename)
        #gameLogQuery = filename.split("/gameData/")[1].split(".maj")[0]
        gameLogID = filename.split("/ultraVigor/")[1].split("&tw=")[0]
        #gameLogQuery = filename.split("/gameData/")[1].split(".maj")[0]
        #gameLogID = filename.split("/gameData/")[1].split("&tw=")[0]
        #print("gameLogQuery",gameLogQuery)
        #print("gameLogID",gameLogID)
        #position = filename.split("tw=")[1][0]
        SHs = []
        finalScores = []
        scores = []
        moves = []
        pointsForGrabs = []
        unconvertedDoraIndicator = []
        convertedDoraIndicator = []
        convertedDora = []
        roundWind = []
        bonusSticks = []
        #print(position)
        playerRatings = source.split('rate="')[1].split('"')[0].split(",")
        for p in range(len(playerRatings)):
            playerRatings[p] = float(playerRatings[p])
        danRanks = source.split('dan="')[1].split('"')[0].split(",")
        for d in range(len(danRanks)):
            danRanks[d] = int(danRanks[d]) 
        lobby = lobbyBallparker(danRanks)
        #lobbyDetermination(danRanks,playerRatings)
        #print("playerRatings",playerRatings)
        #print("danRanks",danRanks)
        #print("lobby",lobby)
        #print(source)
        needMOAR = True
        hands.append([])
        hands[-1] = map(str,source.split("/><INIT"))
        for i in range(1,len(hands[-1])):
            #initial dora
            toAppendDora = int([hands[-1][i].split('seed="')[1].split('"')[0].split(',')[-1]][0])
            unconvertedDoraIndicator.append([toAppendDora])
            convertedDoraIndicator.append([numbaConvert(toAppendDora)])
            convertedDora.append([indicatorCycleConverted(numbaConvert(toAppendDora))])
            roundWind.append([hands[-1][i].split('seed="')[1].split('"')[0].split(',')[0]])
            bonusSticks.append([hands[-1][i].split('seed="')[1].split('"')[0].split(',')[1]])
            #print('dora',dora)
            #
            #print(hands[-1][i])
            SHs.append(0)
            hais = []
            #scores section
            tens = hands[-1][i].split('ten="')[1].split('"')[0].split(",")
            scores.append(tens)
            for j in range(len(scores[-1])):
                scores[-1][j] = int(scores[-1][j])
            #print(scores)
            #SH section
            for x in range(4):
                splitStr = 'hai'+str(x)+'="'
                hai = hands[-1][i].split(splitStr)[1].split('"')[0].split(",")
                for j in range(len(hai)):
                    hai[j] = int(hai[j])
                hais.append(sorted(hai))
            SHs[-1] = hais
            #moves section
            iterThru = hands[-1][i].split("/><")
            iterThru = iterThru[1:]
            agariLocation = -1
            for x in range(len(iterThru)):
                if iterThru[x][:5] == "AGARI" or iterThru[x][:5] == "RYUUK":
                    agariLocation = x
            #print("iterThru",iterThru)
            handEnd = iterThru[agariLocation]
            finalScores.append(handEnd.split('sc="')[1].split('"')[0].split(","))
            iterThru = iterThru[:agariLocation]
            if iterThru[-1][:5] == "AGARI" or iterThru[-1][:5] == "RYUUK":
                iterThru.remove(iterThru[-1])
            #print("handEnd",handEnd)
            #print("iterThru",iterThru)
            moves.append([iterThru,handEnd])
            #print("agariLocation",agariLocation)
            #print("agariPlugged",iterThru[agariLocation])
            #print("finalScores",finalScores)
            #print("===")
            
        
        #q += 1
        #print("=================")
        #print("scores",scores)
        #print("SHs",SHs)
        #print("moves",moves)
        #print("len(moves)",len(moves))
        
        doraChange = "DO"
        drawType = ["T","U","V","W"]
        discType = ["D","E","F","G"]
        stealType = ["N"]
        reachType = ["R"]
        
        moveCount = 0
        handCount = 0
        
        #to know which hand to grab when iterating through based on perspective.
        eastCycle_ = [0,1,2,3]
        southCycle_ = [1,2,3,0]
        westCycle_ = [2,3,0,1]
        northCycle_ = [3,0,1,2]
        
        eastWindCycle_ = ["east","south","west","north"]
        southWindCycle_ = ["south","west","north","east"]
        westWindCycle_ = ["west","north","east","south"]
        northWindCycle_ = ["north","east","south","west"]
        
        fullWindCycle_ = [eastWindCycle_,southWindCycle_,westWindCycle_,northWindCycle_]
        
        windCycle_ = ["east","south","west","north"]

        #you can index by some recalculated value corresponding to the index in this list to get
        #the appropriate perspective based on who is currently acting in the game.
        fullCycle_ = [eastCycle_,southCycle_,westCycle_,northCycle_]
        for i in range(len(moves)):
            
            #that constantly recalculated value for identifying which perspective we're using:
            cycleValue = "undefined" #initialized incorrectly so that if it's used wrong, I'll know.
            
            #print("roundWind[i]",roundWind[i])
            
            if int(roundWind[i][0]) in range(4):
                currentRoundWind = "east"
                currentRoundNumber = i+1
            elif int(roundWind[i][0]) in range(4,8):
                currentRoundWind = "south"
                currentRoundNumber = (i+1)-4
            else: #8,12
                currentRoundWind = "west"
                currentRoundNumber = (i+1)-8
                
            
            bStix = int(bonusSticks[i][0])
            #tenhou game cannot go to north round
            
            unconvertedHands = {}
            for z in range(4):
                unconvertedHands[z] = SHs[i][z][:]
            
            #print("unconvertedHands",unconvertedHands)
            
            convertedHands = {}
            for z in range(4):
                convertedHands[z] = SHs[i][z][:]
                for t in range(len(convertedHands[z])):
                    convertedHands[z][t] = numbaConvert(int(convertedHands[z][t]))
                convertedHands[z] = sorted(convertedHands[z])
            #print("convertedHands",convertedHands)
            
            unconvertedPonds = {}
            for z in range(4):
                unconvertedPonds[z] = []
            
            convertedPonds = {}
            for z in range(4):
                convertedPonds[z] = []
            
            #dora = dora but update on /><DORA
            moveCount = 0
            handCount += 1
        
            #
            reachStatus = {}
            for z in range(4):
                reachStatus[z] = 0
                
            #
            openStatus = {}
            for z in range(4):
                openStatus[z] = 0
                
            #
            openTiles = {}
            for z in range(4):
                openTiles[z] = []
                
            #new thing for whether or not the player discarded the drawn tile or not:
            discDraw = {}
            for z in range(4):
                discDraw[z] = []
            
            for x in range(len(moves[i][0])):
                if x == 0:
                    previousUnconv = -1
                #print("moves[i][0][x][0]",moves[i][0][x][0])
                mFL2 = moves[i][0][x][:2]
                mFL = moves[i][0][x][0] #moveFirstLetter
                #print("mFL",mFL)
                if mFL2 == "DO":
                    toAppend2 = int(moves[i][0][x].split('hai="')[1].split('"')[0])
                    unconvertedDoraIndicator[i].append(int(toAppend2))
                    convertedDoraIndicator[i].append(numbaConvert(toAppend2))
                    convertedDora[i].append(indicatorCycleConverted(numbaConvert(toAppend2)))
                    #print("convertedDoraIndicator",convertedDoraIndicator)
                    #print("convertedDora",convertedDora)
                    #print("toAppend2",toAppend2)
                elif mFL == "R" and int(moves[i][0][x].split('step="')[1].split('"')[0]) == 2:
                    pass
                else:
                    moveCount += 1
                    if mFL in drawType:
                        daMoveType = "draw"
                    elif mFL in discType:
                        daMoveType = "discard"
                    elif mFL == "N":
                        daMoveType = "steal"
                        #print("stupidmFL?",mFL)
                        #print("hannum",i)
                        #print("moonum",x)
                        #print(myProcessTile)
                    elif mFL == "R":
                        daMoveType = "reach"
                    else:
                        daMoveType = "unknown"
                        #print("filename",filename)
                        #print("We got a weird one!")
                        print("THE CRAZY MOVE THAT IS BREAKING EVERYTHING.",moves[i][0][x])
                    
                    
                    rows.append([])
                    rows[-1].append(gameLogID)
                    rows[-1].append(lobby)
                    rows[-1].append(daMoveType)
                    
                    #determining cycleValue for row perspective determination:
                    if mFL in ["T","D"]:
                        cycleValue = 0
                    elif mFL in ["U","E"]:
                        cycleValue = 1
                    elif mFL in ["V","F"]:
                        cycleValue = 2
                    elif mFL in ["W","G"]:
                        cycleValue = 3
                    elif mFL == "N":
                        whoStatement = int(moves[i][0][x].split('who="')[1].split('"')[0])
                        #cycleValue = previousCycleValue
                        cycleValue = whoStatement
                    elif mFL == "R":
                        whoStatement = int(moves[i][0][x].split('who="')[1].split('"')[0])
                        cycleValue = whoStatement
                    else:
                        pass
                        #print("WHAAAAAAAT!?!?!?!?!?")
                    
                    if daMoveType == "steal":
                        parseCall = parse_call(int(moves[i][0][x].split('m="')[1].split('"')[0]))
                    
                    if daMoveType == "steal" and moves[i][0][x-1][0] == "R":
                         if parseCall['type'] != "kan":
                            daTile = int(moves[i][0][x-2][1:]) #unconv
                            myProcessTile = rows[-2][4] #conv
                            openStatus[cycleValue] = 1
                            #print("MEEEEEEELD",parse_call(int(moves[i][0][x].split('m="')[1].split('"')[0])))
                            if parseCall['type'] == "pon":
                                openTiles[cycleValue].append(myProcessTile)
                                openTiles[cycleValue].append(myProcessTile)
                                openTiles[cycleValue].append(myProcessTile)
                            elif parseCall['type'] == "chi":
                                #print(parseCall) #just to make sure chis work
                                openTiles[cycleValue].append(numbaConvert(parseCall['tiles'][0]))
                                openTiles[cycleValue].append(numbaConvert(parseCall['tiles'][1]))
                                openTiles[cycleValue].append(numbaConvert(parseCall['tiles'][2]))
                            elif parseCall['type'] == "kan": #can be drawn... ugh...
                                myProcessTile = numbaConvert(int(moves[i][0][x-1][1:]))
                                daTile = int(moves[i][0][x-1][1:])
                                print("We in here 1",myProcessTile,daTile)
                                #print(parseCall)
                                #print("kan")
                                if moves[i][0][x-1][0] in discType:
                                    #print("kan is stolen")
                                    openStatus[cycleValue] = 1
                                    ecksFour = numbaConvert(int(moves[i][0][x-1][1:]))
                                    openTiles[cycleValue].append(ecksFour)
                                    openTiles[cycleValue].append(ecksFour)
                                    openTiles[cycleValue].append(ecksFour)
                                    openTiles[cycleValue].append(ecksFour)
                                elif moves[i][0][x-1][0] in drawType:
                                    #print("kan is drawn")
                                    ecksFour = numbaConvert(int(moves[i][0][x-1][1:]))
                                    openTiles[cycleValue].append(ecksFour)
                                    openTiles[cycleValue].append(ecksFour)
                                    openTiles[cycleValue].append(ecksFour)
                                    openTiles[cycleValue].append(ecksFour)
                                    # NOT openStatus[cycleValue] = 1, these are always CLOSED KANS
                                elif parseCall['type'] == "none": #looks like this is a DRAWN OPEN KAN ONLY
                                    #need to update these -- 
                                    #daTile = 
                                    #myProcessTile = 
                                    #print(parseCall)
                                    #print("none")
                                    openStatus[cycleValue] = 1
                                    openTiles[cycleValue].append(numbaConvert(int(moves[i][0][x-1][1:])))
                                else:
                                    daTile = "Error1"
                                    myProcessTile =  "Error1"
                                
                    else:    
                        #Determining da tile and da number version of it:
                        if daMoveType in ["draw","discard"]:
                            daTile = int(moves[i][0][x][1:])
                            myProcessTile = numbaConvert(int(moves[i][0][x][1:]))
                        elif daMoveType == "reach":   
                            #reach is just a move so there's no tile so -1 is decent val to maintain column as int
                            reachStatus[cycleValue] = 1
                            daTile = -1
                            myProcessTile = -1
                        elif daMoveType == "steal" and parseCall['type'] != "kan":
                            daTile = int(moves[i][0][x-1][1:])
                            myProcessTile = rows[-2][4]
                            openStatus[cycleValue] = 1
                            #print("MEEEEEEELD",parse_call(int(moves[i][0][x].split('m="')[1].split('"')[0])))
                            if parseCall['type'] == "pon":
                                openTiles[cycleValue].append(myProcessTile)
                                openTiles[cycleValue].append(myProcessTile)
                                openTiles[cycleValue].append(myProcessTile)
                            elif parseCall['type'] == "chi":
                                #print(parseCall) #just to make sure chis work
                                openTiles[cycleValue].append(numbaConvert(parseCall['tiles'][0]))
                                openTiles[cycleValue].append(numbaConvert(parseCall['tiles'][1]))
                                openTiles[cycleValue].append(numbaConvert(parseCall['tiles'][2]))
                        elif parseCall['type'] == "kan": #can be drawn..
                            myProcessTile = numbaConvert(int(moves[i][0][x-1][1:]))
                            daTile = int(moves[i][0][x-1][1:])
                            print("We in here",myProcessTile,daTile)
                            if moves[i][0][x-1][0] in discType:
                                #print("kan is stolen")
                                openStatus[cycleValue] = 1
                                ecksFour = numbaConvert(int(moves[i][0][x-1][1:]))
                                openTiles[cycleValue].append(ecksFour)
                                openTiles[cycleValue].append(ecksFour)
                                openTiles[cycleValue].append(ecksFour)
                                openTiles[cycleValue].append(ecksFour)
                            elif moves[i][0][x-1][0] in drawType:
                                #print("kan is drawn")
                                ecksFour = numbaConvert(int(moves[i][0][x-1][1:]))
                                openTiles[cycleValue].append(ecksFour)
                                openTiles[cycleValue].append(ecksFour)
                                openTiles[cycleValue].append(ecksFour)
                                openTiles[cycleValue].append(ecksFour)
                                # NOT openStatus[cycleValue] = 1, these are always CLOSED KANS
                            elif parseCall['type'] == "none": #looks like this is a DRAWN OPEN KAN ONLY
                                #need to update these -- 
                                #daTile = 
                                #myProcessTile = 
                                #print(parseCall)
                                #print("none omg")
                                openStatus[cycleValue] = 1
                                openTiles[cycleValue].append(numbaConvert(int(moves[i][0][x-1][1:])))
                        else:
                            daTile = "Error2"
                            myProcessTile =  "Error2"
                    
                    #appending to tile column:
                    rows[-1].append(daTile)
                    rows[-1].append(myProcessTile)
                    
                    #list of whether or not he disced drawn tile:
                    if daTile == previousUnconv and daMoveType == "discard":
                        discDraw[cycleValue].append(1)
                    elif daTile != previousUnconv and daMoveType == "discard":
                        discDraw[cycleValue].append(0)
                    else:
                        pass
                    #    
                    rows[-1].append(discDraw[cycleValue][:])
                    #
                    rows[-1].append(handCount) #handNumber
                    rows[-1].append(moveCount) #moveNumber
                    rows[-1].append(currentRoundWind)
                    rows[-1].append(currentRoundNumber)
                    rows[-1].append(bStix)
                    rows[-1].append(unconvertedDoraIndicator[i])
                    rows[-1].append(convertedDoraIndicator[i])
                    rows[-1].append(convertedDora[i])
                    
                    #print("daTile",daTile)
                    #print("myProcessTile",myProcessTile)
                    
                    #Players' hand winds right here (relative to actor):
                    dealerIdentity = moves[i][0][0][0]
                    if dealerIdentity == "T":
                        dealerInt = 0
                    elif dealerIdentity == "U":
                        dealerInt = 1
                    elif dealerIdentity == "V":
                        dealerInt = 2
                    else: #"W"
                        dealerInt = 3
                    #cycleValue is cycleValue
                    #windCycle = ["east","south","west","north"]
                    windCycleMovement = abs(dealerInt-cycleValue)
                    #actorID
                    rows[-1].append(cycleValue)
                    #actorWind
                    #rows[-1].append(windCycle_[windCycleMovement])
                    
                    
                    for j in playerRatings:
                        rows[-1].append(j)
                    for j in danRanks:
                        rows[-1].append(int(j))
                    for j in range(len(scores[i])):
                        rows[-1].append(scores[i][j])
                        
                    ## add currentHands using cyclevalue to adjust perspective on convertedhands dictionary:   
                    #also, add counts for hand values to give the unordered counts of tiles:
                    toCount = [1,2,3,4,5,0,6,7,8,9,21,22,23,24,25,0,26,27,28,29,41,42,43,44,45,0,46,47,48,49,
                              60,70,80,90,100,110,120]
                    handCounts = {}
                    for c in range(4):
                        handCounts[c] = {}
                        for cc in toCount:
                            handCounts[c][cc] = 0
                    
                    #print(q)
                    for c in fullCycle_[cycleValue]:
                        rows[-1].append(convertedHands[c][:])
                        rows[-1].append(unconvertedHands[c][:])
                    
                    tileCountz = {}
                    for c in range(4):
                        tileCountz[c] = [0]*37
                    
                    #count for each tile for each person
                    for c in range(37):
                        for player in fullCycle_[cycleValue]:
                            if c == 4:
                                if 16 in unconvertedHands[player]:
                                    tileCountz[player][4] += 1
                            elif c == 14:
                                if 52 in unconvertedHands[player]:
                                    tileCountz[player][14] += 1
                            elif c == 24:
                                if 88 in unconvertedHands[player]:
                                    tileCountz[player][24] += 1
                            else:
                                tileCountz[player][c] += convertedHands[player].count(toCount[c])
                                
                    #print("tileCountz",tileCountz)
                                    
                    #currentWorkingPoint (took a break)
                    
                    #You next have to append all the kagillion columns of people's individual counts of
                    #each individual tile
                    
                    ####################################
                    ####################################
                    ####################################
                    ####################################
                    ####################################
                    #HAND MUTE
                    ####################################
                    ####################################
                    ####################################
                    ####################################
                    ####################################
#                     for player in fullCycle_[cycleValue]:
#                         for c in range(len(toCount)):
#                             rows[-1].append(tileCountz[player][c])
                    
                    #Next, you have to either keep or change the order of the column headers for those tiles.
                    #'''changed the order'''
                    
                    #print("preRemovalCH_p0",convertedHands[0])
                    #print("preRemovalUH_p0",unconvertedHands[0])
                    
                    #Next next, you have to either append to or remove from the converted and unconverted hands
                    #whatever must be changed, and then sort the hands.
                    
                    #BugSection
                    
                    #bugCheckReach
#                     if myProcessTile == -1:
#                         print(daMoveType)
#                         if daMoveType != "reach":
#                             #print("hannum",i)
#                             p#rint("moonum",x)
#                             print moves[i][0][x]

                    
                    if daMoveType == "draw":
                        convertedHands[fullCycle_[cycleValue][0]].append(myProcessTile)
                        unconvertedHands[fullCycle_[cycleValue][0]].append(daTile)
                        #have to sort the hands as well
                        convertedHands[fullCycle_[cycleValue][0]] = sorted(convertedHands[fullCycle_[cycleValue][0]])
                        unconvertedHands[fullCycle_[cycleValue][0]] = sorted(unconvertedHands[fullCycle_[cycleValue][0]])
                    elif daMoveType == "steal":
                        if parseCall['type'] == "kan":
                            if daTile in unconvertedHands[fullCycle_[cycleValue][0]]:
                                convertedHands[fullCycle_[cycleValue][0]].remove(myProcessTile)
                                unconvertedHands[fullCycle_[cycleValue][0]].remove(daTile)
                            #pass
                        elif parseCall['type'] == "none":
                            if daTile in unconvertedHands[fullCycle_[cycleValue][0]]:
                                convertedHands[fullCycle_[cycleValue][0]].remove(myProcessTile)
                                unconvertedHands[fullCycle_[cycleValue][0]].remove(daTile)
                            #pass
                        else:
                            convertedHands[fullCycle_[cycleValue][0]].append(myProcessTile)
                            unconvertedHands[fullCycle_[cycleValue][0]].append(daTile)
                            #have to sort the hands as well
                            convertedHands[fullCycle_[cycleValue][0]] = sorted(convertedHands[fullCycle_[cycleValue][0]])
                            unconvertedHands[fullCycle_[cycleValue][0]] = sorted(unconvertedHands[fullCycle_[cycleValue][0]])                            
#                             print("myProcessTile",myProcessTile)
#                             print("theMove",move[i][0][x])
                    elif daMoveType == "discard":
                        if daTile in unconvertedHands[fullCycle_[cycleValue][0]]:
                            convertedHands[fullCycle_[cycleValue][0]].remove(myProcessTile)
                            unconvertedHands[fullCycle_[cycleValue][0]].remove(daTile)
                        #don't need to sort, because something was deleted in place and order is preserved.
                    elif daMoveType == "reach":
                        pass
                        # here for readability I guess, but yeah no tiles have changed here so nothing happens.
                    else:
                        #pass
                        print("Error!!!!!!!!!! -- weird move type!?")
                        
                        
                    #print("postRemovalCH_p0",convertedHands[0])
                    #print("postRemovalUH_p0",unconvertedHands[0])                        
                    
                    
                    
                    ###############
                    # I suppose I'll address ponds now.
                    ###############
                    
                    if daMoveType == "discard":
                        unconvertedPonds[cycleValue].append(daTile)
                        convertedPonds[cycleValue].append(myProcessTile)
                        
                    #print("unconvertedPonds",unconvertedPonds)
                    #print("convertedPonds",convertedPonds)
                    
                    for player in fullCycle_[cycleValue]:
                        rows[-1].append(unconvertedPonds[player][:])
                        rows[-1].append(convertedPonds[player][:])
                        
                    
                    
                    tileCountzPond = {}
                    for c in range(4):
                        tileCountzPond[c] = [0]*37
                    
                    #count for each tile for each person
                    for c in range(37):
                        for player in fullCycle_[cycleValue]:
                            if c == 4:
                                if 16 in unconvertedPonds[player]:
                                    tileCountzPond[player][4] += 1
                            elif c == 14:
                                if 52 in unconvertedPonds[player]:
                                    tileCountzPond[player][14] += 1
                            elif c == 24:
                                if 88 in unconvertedPonds[player]:
                                    tileCountzPond[player][24] += 1
                            else:
                                tileCountzPond[player][c] += convertedPonds[player].count(toCount[c])
                                
                    #print("tileCountzPond",tileCountzPond)
                    
                    ####################################
                    ####################################
                    ####################################
                    ####################################
                    ####################################
                    #POND MUTE
                    ####################################
                    ####################################
                    ####################################
                    ####################################
                    ####################################
                    
#                     for player in fullCycle_[cycleValue]:
#                         for c in range(len(toCount)):
#                             rows[-1].append(tileCountzPond[player][c])
                    #######
                    ##Appending reach statuses even though it's declared way earlier.
                    for player in fullCycle_[cycleValue]:
                        rows[-1].append(reachStatus[player])
                    
                    ###############
                    # Addressing openness
                    ###############
                    for player in fullCycle_[cycleValue]:
                        rows[-1].append(openStatus[player])
                        
                    for player in fullCycle_[cycleValue]:
                        rows[-1].append(sorted(openTiles[player][:]))
                        
                    for player in fullCycle_[cycleValue]:
                        rows[-1].append(np.sum(tileCountzPond[player]))
                        
                    #######################################
                    #Tenpai calc
                    ###################################
                        
                    #########################################
                    # Pond status feature engineering????
                    #########################################
                    
                    #maybe after make dataframe actually.
                    
                    
                    #who is acting
                    rows[-1].append(cycleValue)
                    
                    #
                    previousUnconv = daTile
                    previousConv = myProcessTile
                    
                    #Unconv = 
                    #Conv = 
                    
                    #update cycle for reference ability earlier on.
                    
                    previousCycleValue = cycleValue
                    cycleValue = "reset"
                    
#                     df = pd.DataFrame(data=rows,columns=columnHeaders)
#                     dfList.append(df.copy())
#                     df = 0



######################################
# column where 1 if discard is same as what was drawn last move, 0 if else.
######################################


# for i in range(len(rows)):
#     try:
#         if rows[i][3] == rows[i-1][3] and rows[i][2] == "discard" and rows[i-1][2] == "draw":
#             rows[i].append(1)
#         else:
#             rows[i].append(0)
#     except:
#         rows[i].append(0)
        

                    
# time_end = time.time()

# print("=================")
# print("=================")
# print("=================")
# print("=================")
# print("=================")

# print(time_end-time_start)


#dfFull = pd.concat(dfList)
#dfFull.to_csv("tesqFull.csv")
df = pd.DataFrame(data=rows,columns=columnHeaders)
# for i in df["actorHandConverted"].values:
#     print len(i)
#     print i

df.to_csv("goodPlz/houou.csv")

('filename', '/home/tpeach/Desktop/h/mahjongDataML-and-Bot/ultraVigor/2016082221gm-00a9-0000-ebc65731&tw=3.maj')
('We in here', 46, 92)
('We in here', 25, 54)
contained bye so dropped it
('filename', '/home/tpeach/Desktop/h/mahjongDataML-and-Bot/ultraVigor/2016082113gm-00a9-0000-44024277&tw=0.maj')
('filename', '/home/tpeach/Desktop/h/mahjongDataML-and-Bot/ultraVigor/2016082113gm-00a9-0000-2507b92a&tw=3.maj')
('filename', '/home/tpeach/Desktop/h/mahjongDataML-and-Bot/ultraVigor/2016082112gm-00a9-0000-de3c9814&tw=2.maj')
('filename', '/home/tpeach/Desktop/h/mahjongDataML-and-Bot/ultraVigor/2016082112gm-00a9-0000-627bf9c1&tw=0.maj')
contained bye so dropped it
('filename', '/home/tpeach/Desktop/h/mahjongDataML-and-Bot/ultraVigor/2016082110gm-00a9-0000-53605ba9&tw=2.maj')
('filename', '/home/tpeach/Desktop/h/mahjongDataML-and-Bot/ultraVigor/2016082110gm-00a9-0000-4daada0a&tw=1.maj')
('filename', '/home/tpeach/Desktop/h/mahjongDataML-and-Bot/ultraVigor/2016082109gm-00a9-0000-1d1cef34&tw=1.

In [84]:
print(df.shape)

(44067, 60)


In [85]:
df["discDrawn"].values

array([[], [0], [], ..., [0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1],
       [0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1],
       [0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0]], dtype=object)

In [86]:
dfDiscard = df[df["moveType"]=="discard"]

In [87]:
lengths = []
indexes = []
valz = dfDiscard["actorHandConverted"].values
# for i in range(len(valz)):
#     lengths.append(len(valz[i]))
#     lengths.append(i)
    
for i in range(len(valz)):
    if len(valz[i]) != 14:
        print i

21179
21183


In [88]:
dfDiscard[["actorHandConverted","gameLogID","handNumber"]].values[194]

array([[4, 4, 4, 22, 24, 24, 25, 25, 25, 46, 46, 46, 110, 110],
       '2016082221gm-00a9-0000-ebc65731', 3], dtype=object)

In [ ]:
parse_call(13312)

In [ ]:
#df.info(max_cols=360)

In [ ]:
#print(filename)
#print("/><BYE" in source)
#print(source.split("/><INIT")[1].split("/><INIT")[0])
# print("========")
# print("========")

# idfk = []

# for i in range(len(moves[0][0])):
#     idfk.append(rows[i][3])
    
# print(idfk)

# idfk2 = []

# for i in range(len(moves[0][0])):
#     toAppendNao = numbaConvert(rows[i][3])
#     idfk2.append(toAppendNao)
    
# print(idfk2)
#print(rows[0][0])
#print(rows[4000])
#print(len(rows[0]))

In [ ]:
# discGames = 0
# totalInFolder = 28891.

# for filename in glob.glob(os.path.join(renpath, '*.maj')):
#     source = open(filename,'r').read()
#     if "/><BYE" not in source:
#         discGames += 1
# print(discGames/totalInFolder)

# #If we remove disconnects we lose roughly 10% of the games.  (Not bad actually!)

# #I don't want games where the integrity of the play is in question.